In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [2]:
!pip install tf-keras tensorflow==2.15.0

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [4]:
max_features = 10000
max_len = 500

In [5]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 2s 0us/step


In [6]:
X_all = np.concatenate([X_train, X_test])
y_all = np.concatenate([y_train, y_test])

In [7]:
X_train = X_all[:40000]
y_train = y_all[:40000]

X_test = X_all[40000:]
y_test = y_all[40000:]

In [8]:
word_index = imdb.get_word_index()
index_to_word = {index + 3 : word for word, index in word_index.items()}
index_to_word[0], index_to_word[1], index_to_word[2] = "<PAD>", "<START>", "<UNK>"

1641221/1641221 [==============================] - 1s 1us/step


In [9]:
def decode(encoded_review):
	return " ".join([index_to_word.get(word, "<UNK>") for word in encoded_review])

In [10]:
X_train_text = [decode(review) for review in X_train]
X_test_text = [decode(review) for review in X_test]

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
X_train_encodings = tokenizer(X_train_text, padding="max_length", truncation=True, max_length=max_len, return_tensors="tf")
X_test_encodings = tokenizer(X_test_text, padding="max_length", truncation=True, max_length=max_len, return_tensors="tf")

In [13]:
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [14]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [15]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy("accuracy")
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [16]:
history = model.fit(
	{
		"input_ids": X_train_encodings["input_ids"],
		"attention_mask": X_train_encodings["attention_mask"]
	},
	y_train,
	validation_data=(
		{
			"input_ids": X_test_encodings["input_ids"],
			"attention_mask": X_test_encodings["attention_mask"]
		},
		y_test
	),
	epochs=4,
	batch_size=32
)

Epoch 1/4


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1250/1250 [==============================] - 2304s 2s/step - loss: 0.7252 - accuracy: 0.5000 - val_loss: 0.6932 - val_accuracy: 0.5027
Epoch 2/4
1250/1250 [==============================] - 2208s 2s/step - loss: 0.6935 - accuracy: 0.5038 - val_loss: 0.6938 - val_accuracy: 0.5027
Epoch 3/4
1250/1250 [==============================] - 2228s 2s/step - loss: 0.6936 - accuracy: 0.4999 - val_loss: 0.6937 - val_accuracy: 0.5027
Epoch 4/4
1250/1250 [==============================] - 2226s 2s/step - loss: 0.6935 - accuracy: 0.5034 - val_loss: 0.6932 - val_accuracy: 0.4973


In [17]:
results = model.evaluate(
	{
		"input_ids": X_test_encodings["input_ids"],
		"attention_mask": X_test_encodings["attention_mask"]
	},
	y_test
)

313/313 [==============================] - 182s 582ms/step - loss: 0.6932 - accuracy: 0.4973


In [18]:
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")

Test Loss: 0.693188488483429
Test Accuracy: 0.49729999899864197


In [19]:
model.save_pretrained("./models")
tokenizer.save_pretrained("./models")

('./models/tokenizer_config.json',
 './models/special_tokens_map.json',
 './models/vocab.txt',
 './models/added_tokens.json')